In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
  print('Simply select "GPU" in the Accelerator drop-down in Notebook Settings')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import requests
url = "https://raw.githubusercontent.com/peterkabai/tensorFlow/master/textFiles/sorcerersStone.txt"
data = requests.get(url).text

In [0]:
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

In [0]:
# the number of sequential characters to sample
length_of_input = 12

# maps each character to a number and each number to a character
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# create training sequences and corresponding labels
import numpy as np
X = []
y = []
for i in range(0, len(data)-length_of_input-1, 1):
  X.append([char_to_ix[ch] for ch in data[i:i+length_of_input]])
  y.append([char_to_ix[ch] for ch in data[i+1:i+length_of_input+1]])

# reshapes the data
X_modified = np.reshape(X, (len(X), length_of_input))
y_modified = np.reshape(y, (len(y), length_of_input))


In [0]:
tf.reset_default_graph()
import random
import warnings
warnings.filterwarnings("ignore")

# sets hyperparameters to mimic the Karpathy code
n_neurons = 100
num_batches = 10
iterations = 200
n_layers = 2
learning_rate = 0.001
sequence_to_use = 0
num_sequences = X_modified.shape[0]
epochs_run = 0

# X has any number of batches and chars, and vocab_size due to one-hot encoding
X = tf.placeholder(tf.float32, [None, None, vocab_size])

# y has any number of batches, and length_of_input characters
y = tf.placeholder(tf.int32, [None, length_of_input])

# more TensorFlow stuff defined here
layers = [tf.contrib.rnn.LSTMBlockCell(num_units=n_neurons) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
logits = tf.layers.dense(outputs, vocab_size)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
probs = tf.nn.softmax(logits)

In [0]:
epoch_count = 0
# function to front the characters that match an array of indices
def print_chars(indexArray):
    string = ""
    for c in indexArray:
        if (c != None):
            string += ix_to_char.get(c)
    print(string)
    
startChars = [
    "A","B","C","D","E","F","G","H","I",
    "J","K","L","M","N","O","P","Q","R",
    "S","T","U","V","W","X","Y","Z"
]
startSize = len(startChars)

In [0]:
init = tf.global_variables_initializer()
print_every = 100
with tf.Session() as sess:
    init.run()
    
    print("Encoding started...")
    X_encoded = tf.one_hot(X_modified, vocab_size)
    X_encoded = X_encoded.eval()
    print("Encoding done!")
    
    # epoch loop starts here
    while (True):
        
        if (epoch_count % print_every == 0):
            # creates a seed to start the string from
            pred_indices = [char_to_ix.get(startChars[random.randint(0,startSize-1)])]
        
        # each iteration is one new character
        for iteraton in range(0, iterations):
            
            # creates batches, each row is an array of sequential characters
            in_indices = []
            out_indices = []
            for batch in range(0, num_batches):
            
                # if we run out of sequences, the sequence to use returns to 0
                if (sequence_to_use >= num_sequences):
                    sequence_to_use = 0
                    
                # in and out indices are appended to
                # 'sequence_to_use' is incrimented to get the next sequence when re-run
                in_indices.append(X_encoded[sequence_to_use])
                out_indices.append(y_modified[sequence_to_use])
                sequence_to_use += 1
            
            # run the trainining op
            sess.run(training_op, feed_dict={X: np.asarray(in_indices), y: np.asarray(out_indices)})
        
            if (epoch_count % print_every == 0):
                # one hot encode the prediction indices
                pred_encoded = tf.one_hot(np.asarray(pred_indices), vocab_size).eval()
                
                # get predictions as probabilities
                predictions = sess.run(probs, feed_dict={X: np.asarray([pred_encoded])})
                
                # take the probabilities from the last character
                # pick the next index using the probabilities
                ix = np.random.choice(range(vocab_size), p=(predictions[0][-1]).ravel())
                
                # add to the array of indices
                pred_indices.append(ix)
                
        if (epoch_count % print_every == 0):
            # print the string every epoch for the first 'iterations_to_print' epochs
            print("\nEpoch:", epoch_count,"---------------------------------------------")
            print_chars(pred_indices)
        
        # increment the epoch count used to see when to print the results
        epoch_count = epoch_count + 1

Encoding started...
Encoding done!

Epoch: 0 ---------------------------------------------
NsWDpoxU,F1gn,6jtmEuw  , tey   tt    tcsBot id mhh lcseansdaimBrd tds  .wn rwesh  in  reyiieer tlahet o etreoaimud ereneust.edrssosdidtnt tuyndteasy rsei ity r ihd rdnolaafh  y nDloy uyuoeoyl rr'aDo hh

Epoch: 100 ---------------------------------------------
At mile glaim," 
Professor Fbrong dorn and nape it as an up a poind in in on the oxle romess was palled up ever at fon Potter is is is ands. 
Gleffants, but it's pikering drice?" se gate she reeping 

Epoch: 200 ---------------------------------------------
The bully," sait Harry. 
"At was ening if their wordetionly a fack. He robecing his head and sured... he boked him from with thing to that Peevenge befo the mirecting reoply at her a spen the around t

Epoch: 300 ---------------------------------------------
Hermione afred into pale sweetes sout over expects. Sinside as to gran's onved out of me so tiekhctory's going to steared, but I wen